# **Competition**: American Sign Language Recognition

---
Conoscere l'Intelligenza Artificiale



# Importiamo quello che ci serve

In [ ]:
import numpy as np
import pandas as pd

import os

import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPool2D,Dropout,Input
import matplotlib.pyplot as plt
import seaborn as sns
from keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%pip install livelossplot
from livelossplot import PlotLossesKeras
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from PIL import Image
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow import keras as tfk

# Carichiamo i Dataset

In [ ]:
train_df = pd.read_csv("https://raw.githubusercontent.com/alberto-rota/Conoscere-l-Intelligenza-Artificiale/main/sign_mnist_train.csv",index_col=0)[::5].reset_index()
test_df = pd.read_csv("https://raw.githubusercontent.com/alberto-rota/Conoscere-l-Intelligenza-Artificiale/main/sign_mnist_test.csv",index_col=0)[::5].reset_index()
train_df.head(TODO)

In [ ]:
num2lett = {i:chr(65 + i) if i < 9 else chr(66 + i) for i in range(25)}
lett2num = {value: key for key, value in num2lett.items()}
train_df["target"] = train_df["label"].map(lett2num)
test_df["target"] = test_df["label"].map(lett2num)
train_df.head(40)

In [ ]:
train_target = train_df['target']
test_target  = test_df['target']

X_train = train_df.drop(['label','target','index'],axis=1)
X_train = X_train.values.reshape(-1,28,28,1)
X_train = X_train/255
X_test  = test_df.drop(['label','target','index'],axis=1)
X_test  = X_test.values.reshape(-1,28,28,1)
X_test = X_test/255

lb = LabelBinarizer()
y_train = lb.fit_transform(train_target)
y_test  = lb.fit_transform(test_target)

# Quanti dati abbiamo per training? Quanti per test? Proporzione?

In [ ]:
print("Dati per training:",len(X_train),"immagini di dimensione",X_train.shape[1],"x",X_train.shape[2])
print("Dati per test:",len(X_test),"immagini di dimensione",X_test.shape[1],"x",X_test.shape[2])

# Visualizziamo alcune immagini

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
y_train_categorical = lb.inverse_transform(y_train)
r = np.random.randint(0, len(X_train)+1, 4)

axs[0, 0].imshow(X_train[r[0], :, :], cmap='gray')
axs[0, 0].set_title("Lettera "+num2lett[np.where(y_train[r[0]])[0][0]])

axs[0, 1].imshow(X_train[r[1], :, :], cmap='gray')
axs[0, 1].set_title("Lettera "+num2lett[np.where(y_train[r[1]])[0][0]])

axs[1, 0].imshow(X_train[r[2], :, :], cmap='gray')
axs[1, 0].set_title("Lettera "+num2lett[np.where(y_train[r[2]])[0][0]])

axs[1, 1].imshow(X_train[r[3], :, :], cmap='gray')
axs[1, 1].set_title("Lettera "+num2lett[np.where(y_train[r[3]])[0][0]])

plt.tight_layout()
plt.show()

# Controlliamo ...
![reference](https://www.startasl.com/wp-content/uploads/asl-alphabet_wallpaper_1920x1080.png)

In [ ]:
# Questa cella non fa niente ma DEVE essere eseguita
def draw_neural_net(model, left =.1, right=.9, bottom=.1, top=.9):
    '''
    Draw a neural network cartoon using matplotilb.

    :param ax: matplotlib.axes.Axes, the axes on which to plot the cartoon
    :param left: float, the center of the leftmost node(s) will be placed here
    :param right: float, the center of the rightmost node(s) will be placed here
    :param bottom: float, the center of the bottommost node(s) will be placed here
    :param top: float, the center of the topmost node(s) will be placed here
    :param layer_sizes: list of int, list containing the number of nodes in each layer
    '''
    real_input_size=model.input_shape[-1]
    layer_sizes = [50]#
    for layer in model.layers:
        # Check if layer is a Dense layer (ignoring input and output layers for simplicity)
        if isinstance(layer, tf.keras.layers.Dense):
            layer_sizes.append(layer.units)
    v_spacing = (top - bottom)/float(max(layer_sizes))
    h_spacing = (right - left)/float(len(layer_sizes) - 1)
    fig = plt.figure(figsize=(20,10))
    ax = fig.gca()
    ax.axis('off')
    ax.set_aspect('equal')
    # Nodes

    for n, layer_size in enumerate(layer_sizes):
        layer_top = v_spacing*(layer_size - 1)/2. + (top + bottom)/2.
        for m in range(layer_size):
            circle = plt.Circle((n*h_spacing + left, layer_top - m*v_spacing), v_spacing/4.,
                                color='w', ec='k', zorder=4)
            ax.add_artist(circle)
            # Annotation for the layer number
            if m == layer_size - 1:
                if n == 0:
                    plt.text(n*h_spacing + left, layer_top - m*v_spacing - v_spacing*4, f'{real_input_size} N',
                         ha='center', va='center')
                else:
                    plt.text(n*h_spacing + left, layer_top - m*v_spacing - v_spacing*4, f'{layer_size} N',
                            ha='center', va='center')
    #Edges
    for n, (layer_size_a, layer_size_b) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
        layer_top_a = v_spacing*(layer_size_a - 1)/2. + (top + bottom)/2.
        layer_top_b = v_spacing*(layer_size_b - 1)/2. + (top + bottom)/2.
        for m in range(layer_size_a):
            for o in range(layer_size_b):
                if np.random.uniform(0,1)<0.04:
                    line = plt.Line2D([n*h_spacing + left, (n + 1)*h_spacing + left],
                                    [layer_top_a - m*v_spacing, layer_top_b - o*v_spacing], c='k',linewidth=0.25,alpha=0.5)
                    ax.add_artist(line)
    plt.title("Rete Neurale")
    plt.show()

def reinitialize(model):
    for l in model.layers:
        if hasattr(l,"kernel_initializer"):
            l.kernel.assign(l.kernel_initializer(tf.shape(l.kernel)))
        if hasattr(l,"bias_initializer"):
            l.bias.assign(l.bias_initializer(tf.shape(l.bias)))
        if hasattr(l,"recurrent_initializer"):
            l.recurrent_kernel.assign(l.recurrent_initializer(tf.shape(l.recurrent_kernel)))


# Costruiamo la Rete Neurale

In [ ]:
model = Sequential()
LAYERS = [
    # Input --> NON CAMBIARE!
    Input(784),  # Calcolare quanti neuroni in input e sostituire con TODO
    #
    # Dense(TODO, activation='linear'),
    # Dense(TODO, activation='tanh'),
    #TODO: Aggiungiamo  layers come questi sopra. Quanti layer? Quanti neuroni? Quale activation function?
    #...

    # Output --> NON CAMBIARE!
    Dense(24, activation='softmax')
]
for l in LAYERS:
    model.add(l)
draw_neural_net(model)

# Quanti parametri?

In [ ]:
model.summary()

# Training!

In [ ]:
###################################################################
LEARNING_RATE = TODO   # Esempio: 1e-3
BATCH_SIZE    = TODO   # Esempio: 8        # TODO: Cambiare questi valori!
EPOCHE        = TODO   # Esempio: 500
###################################################################

X_train = X_train.reshape(-1,28*28)
X_test = X_test.reshape(-1,28*28)
reinitialize(model)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics='accuracy'
)
history = model.fit(X_train,y_train,
                    validation_data=(X_test,y_test),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHE,
                    shuffle=True,
                    callbacks=[PlotLossesKeras()],
                    verbose=0
                    )

In [ ]:
predictions=model.predict(X_test,verbose=0)
predictions = np.argmax(predictions,axis=1).astype(int)
test_accuracy = accuracy_score(np.argmax(y_test,axis=1).astype(int), predictions)
print(f'Accuratezza sul Dataset di test: {round(test_accuracy, 4)*100}%')

predictions=model.predict(X_train,verbose=0)
predictions = np.argmax(predictions,axis=1).astype(int)
train_accuracy = accuracy_score(np.argmax(y_train,axis=1).astype(int), predictions)
print(f'Accuratezza sul Dataset di training: {round(train_accuracy, 4)*100}%')

# Overfitting?

# Proviamolo in tempo reale!

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Ok';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

while True:
    filename = take_photo()
    image_array = np.array(Image.open(filename).convert('L').resize((28, 28)))
    prediction = model.predict(np.expand_dims(image_array.flatten(),0),verbose=0)
    prediction = num2lett[np.argmax(prediction,axis=1).astype(int)[0]]
    plt.figure()
    plt.imshow(image_array,cmap="gray")
    plt.title(prediction)
    plt.show()